In [1]:
import requests
import json
import pandas as pd
import urllib.request

import soccerfield

import dash
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'notebook'

import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
def load_json(url):
    response = requests.get(url)
    return response.json()

In [4]:
url_WC_2023 = 'https://raw.githubusercontent.com/statsbomb/open-data/master/data/matches/72/107.json'
json_data_2023 = load_json(url_WC_2023)

In [5]:
url_WC_2022 = 'https://raw.githubusercontent.com/statsbomb/open-data/master/data/matches/43/106.json'
json_data_2022 = load_json(url_WC_2022)

In [6]:
json_data_2023

[{'match_id': 3904629,
  'match_date': '2023-08-16',
  'kick_off': '13:00:00.000',
  'competition': {'competition_id': 72,
   'country_name': 'International',
   'competition_name': "Women's World Cup"},
  'season': {'season_id': 107, 'season_name': '2023'},
  'home_team': {'home_team_id': 1205,
   'home_team_name': "Australia Women's",
   'home_team_gender': 'female',
   'home_team_group': None,
   'country': {'id': 14, 'name': 'Australia'},
   'managers': [{'id': 408,
     'name': 'Tony Gustavsson',
     'nickname': None,
     'dob': '1973-08-14',
     'country': {'id': 220, 'name': 'Sweden'}}]},
  'away_team': {'away_team_id': 865,
   'away_team_name': "England Women's",
   'away_team_gender': 'female',
   'away_team_group': None,
   'country': {'id': 68, 'name': 'England'},
   'managers': [{'id': 45,
     'name': 'Sarina Glotzbach-Wiegman',
     'nickname': 'Sarina Wiegman',
     'dob': '1969-10-26',
     'country': {'id': 160, 'name': 'Netherlands'}}]},
  'home_score': 1,
  'away_

In [7]:
def get_teams(match_id):
    home = [match['home_team']['home_team_name'] for match in json_data_2023 if match['match_id'] == match_id][0]
    away = [match['away_team']['away_team_name'] for match in json_data_2023 if match['match_id'] == match_id][0]
    return home, away

In [8]:
Japan_matches_2023 = []
for item in json_data_2023:
    if item['home_team']['home_team_id'] == 1210:
        Japan_matches_2023.append(item['match_id'])
    if item['away_team']['away_team_id'] == 1210:
        Japan_matches_2023.append(item['match_id'])

In [9]:
url_WC_2019 = 'https://raw.githubusercontent.com/statsbomb/open-data/master/data/matches/72/30.json'
json_data_2019 = load_json(url_WC_2019)

In [10]:
Japan_matches_2019 = []
for item in json_data_2019:
    if item['home_team']['home_team_id'] == 1210:
        Japan_matches_2019.append(item['match_id'])
    if item['away_team']['away_team_id'] == 1210:
        Japan_matches_2019.append(item['match_id'])

In [11]:
position_dict = {1:(10, 40), 
                 2:(25, 72), 3:(25, 56), 4:(25, 40), 5:(25, 24), 6:(25, 8),
                 7:(42.5, 72), 9:(42.5, 56), 10:(42.5, 40), 11:(42.5, 24), 8:(42.5, 8),
                 12:(60, 72), 13:(60, 56), 14:(60, 40), 15:(60, 24), 16:(60, 8),
                 17:(77.5, 72), 18:(77.5, 56), 19:(77.5, 40), 20:(77.5, 24), 21:(77.5, 8),
                 25:(88.75, 40), 22:(100, 56), 23:(100, 40), 24:(100, 24)}

In [124]:
def plot_match(match_id, team_nation, opponent_nation):
    url = f'https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/{match_id}.json'
    match_events = load_json(url)
    team = f"{team_nation} Women's"
    opponent = f"{opponent_nation} Women's" 
    
    starting_XI = [event for event in match_events if event['team']['name'] == team and event['type']['id'] == 35]
    position_ids = [player['position']['id'] for player in starting_XI[0]['tactics']['lineup']]
    
    tactic_XI = [event for event in match_events if event['team']['name'] == team and event['type']['id'] == 36]
    
    team_events = [event for event in match_events if event['team']['name'] == team]
    
    goal_events = [event for event in match_events if event['type']['id'] == 16 and 
            event['team']['name'] == team and event['shot']['outcome']['name'] == 'Goal']
    no_goal_events = [event for event in match_events if event['type']['id'] == 16 and 
            event['team']['name'] == team and event['shot']['outcome']['name'] != 'Goal']
    
    oppo_goal_events = [event for event in match_events if event['type']['id'] == 16 and 
            event['team']['name'] == opponent and event['shot']['outcome']['name'] == 'Goal']
    oppo_no_goal_events = [event for event in match_events if event['type']['id'] == 16 and 
            event['team']['name'] == opponent and event['shot']['outcome']['name'] != 'Goal']
    
    goal_seq = {}
    for event in goal_events:
        before_goal_events = team_events[team_events.index(event) - 4:team_events.index(event)+1]
        before_goal_events= [e for e in before_goal_events if 'location' in e]
        goal_seq[event['index']] = before_goal_events
        
    no_goal_seq = {}
    for event in no_goal_events:
        before_no_goal_events = team_events[team_events.index(event) - 4:team_events.index(event)+1]
        before_no_goal_events= [e for e in before_no_goal_events if 'location' in e]
        no_goal_seq[event['index']] = before_no_goal_events
    
    team_carry = [event for event in match_events if event['type']['id'] == 43 and
                event['team']['name'] == team and event['duration'] > 3.5]        
            
    dispo_events = [event for event in match_events if event['team']['name'] == opponent and
               event['type']['id'] == 3]
    
    opponent_carry = [event for event in match_events if event['type']['id'] == 43 and
                event['team']['name'] == opponent and event['duration'] > 3.5]
        
    field_layout = soccerfield.get_layout()
    field_layout.update(legend=dict(x=0.85, y=1))

    fig = go.Figure(layout=field_layout)
    fig.update_layout(title = dict(text = f'{team} vs {opponent}', x =0.5, y =0.9), 
                      title_font=dict(size=20, family='Arial', color='black'))

    # Starting lineups
    fig.add_trace(go.Scatter(
        x = [position_dict[i][0] for i in position_ids],
        y = [position_dict[i][1] for i in position_ids],
        mode='markers',
        name = 'starting XI',
        marker=dict(size=8, symbol = 'circle', color='lightblue',
            line = dict(width = 1, color = 'darkblue')),
    ))
    
    # Tactical shifts
    if len(tactic_XI) != 0:
        for tac in tactic_XI:
            time = str(tac['minute']) + ':' + str(tac['second'])
            tac_ids = [player['position']['id'] for player in tac['tactics']['lineup']]
            if tac_ids != position_ids:
                fig.add_trace(go.Scatter(
                    x = [position_dict[i][0] for i in tac_ids],
                    y = [position_dict[i][1] for i in tac_ids],
                    mode='markers',
                    name = f'tactical shit - {time}',
                    marker=dict(size=8, symbol = 'circle', color='lightblue', opacity=0.5,
                        line = dict(width = 1, color = 'steelblue')),
                    showlegend = True, visible='legendonly'
                ))

    # Shot with no goal - team
    fig.add_trace(go.Scatter(
        x = [event['location'][0] for event in no_goal_events],
        y = [event['location'][1] for event in no_goal_events],
        legendgroup = 'no goal shots',
        name = 'team shot w/ no goal',
        mode='markers',
        marker=dict(size=5, symbol = 'circle', color='darkgoldenrod')
    ))

    # Shot with goals - team
    fig.add_trace(go.Scatter(
        x = [event['location'][0] for event in goal_events],
        y = [event['location'][1] for event in goal_events],
        legendgroup = 'goal shots',
        name = 'team shot w/ goal',
        mode='markers',
        marker=dict(size=6, symbol = 'circle', color='purple')
    ))

    # Shot with no goal - oppo
    fig.add_trace(go.Scatter(
        x = [120-event['location'][0] for event in no_goal_events],
        y = [event['location'][1] for event in no_goal_events],
        legendgroup = 'oppo no goal shots', visible='legendonly',
        name = 'opponent shot w/ no goal',
        mode='markers',
        marker=dict(size=5, symbol = 'circle', color='darkgoldenrod')
    ))
    
    # Shot with goals - oppo
    fig.add_trace(go.Scatter(
        x = [120-event['location'][0] for event in oppo_goal_events],
        y = [event['location'][1] for event in oppo_goal_events],
        legendgroup = 'oppo goal shots', visible='legendonly',
        name = 'opponent shot w/ goal',
        mode='markers',
        marker=dict(size=6, symbol = 'circle', color='purple')
    ))

    # Moves before shot with goals
    for key, seq in goal_seq.items():
        fig.add_trace(go.Scatter(
            x = [event['location'][0] for event in seq[:-1]],
            y = [event['location'][1] for event in seq[:-1]],
            legendgroup = 'goal shots',
            showlegend = False,
            mode='markers',
            marker=dict( size=6, symbol = 'circle', color='purple', opacity=0.3)
        ))
    for key, seq in goal_seq.items():
        fig.add_trace(go.Scatter(
            x=[event['location'][0] for event in seq],  
            y=[event['location'][1] for event in seq], 
            legendgroup = 'goal shots',
            showlegend = False,
            mode='lines',
            line=dict(color='purple', width = 0.25)
        ))

    # Moves before shot with no goal
    for key, seq in no_goal_seq.items():
        fig.add_trace(go.Scatter(
            x = [event['location'][0] for event in seq[:-1]],
            y = [event['location'][1] for event in seq[:-1]],
            legendgroup = 'no goal shots',
            showlegend = False,
            mode='markers',
            marker=dict( size=5, symbol = 'circle', color='tan', opacity=0.3)
        ))
    for key, seq in no_goal_seq.items():
        fig.add_trace(go.Scatter(
            x=[event['location'][0] for event in seq],  
            y=[event['location'][1] for event in seq], 
            legendgroup = 'no goal shots',
            showlegend = False,
            mode='lines',
            line=dict(color='tan', width = 0.18)
        ))    

    # Opponent disposession
    fig.add_trace(go.Scatter(
        x = [120-event['location'][0] for event in dispo_events],
        y = [event['location'][1] for event in dispo_events],
        name = 'opponent disposession',
        mode='markers',
        marker=dict(size=6, symbol = 'x', color='darkseagreen', opacity=0.6)
    ))
    
    # Team carry
    fig.add_trace(go.Scatter(x = [None], y = [None], legendgroup = 'team_carry', name = 'team_carry',
                            mode='lines', line=dict(color='orchid', width = 0.4, dash = 'dash'), visible='legendonly'))
    
    for carry in team_carry:
        fig.add_trace(go.Scatter(
            x = [carry['location'][0], carry['carry']['end_location'][0]],
            y = [carry['location'][1], carry['carry']['end_location'][1]],
            legendgroup = 'team_carry',
            showlegend = False, visible='legendonly',
            mode='lines',
            line=dict(color='orchid', width = 0.3, dash = 'dash')
        ))    
        
    # Opponent carry
    fig.add_trace(go.Scatter(x = [None], y = [None], legendgroup = 'opponent_carry', name = 'opponent_carry',
                            mode='lines', line=dict(color='darkgrey', width = 0.4, dash = 'dash')))
    
    for carry in opponent_carry:
        fig.add_trace(go.Scatter(
            x = [120-carry['location'][0], 120-carry['carry']['end_location'][0]],
            y = [carry['location'][1], carry['carry']['end_location'][1]],
            legendgroup = 'opponent_carry',
            showlegend = False,
            mode='lines',
            line=dict(color='darkgrey', width = 0.3, dash = 'dash')
        ))

    fig.show()    

In [13]:
for match in Japan_matches_2023:
    print(match)
    home, away = get_teams(match)
    print(home+ ' vs '+away)

3893822
Japan Women's vs Spain Women's
3902239
Japan Women's vs Sweden Women's
3901734
Japan Women's vs Norway Women's
3893805
Japan Women's vs Costa Rica Women's
3893793
Zambia Women's vs Japan Women's


In [122]:
plot_match(3893822, 'Japan', 'Spain')

In [89]:
plot_match(3893822, 'Spain', 'Japan')

In [125]:
plot_match(3902239, 'Japan', 'Sweden')

In [91]:
plot_match(3902239,'Sweden','Japan')

In [90]:
plot_match(3901734, 'Norway', 'Japan')

In [86]:
plot_match(3901734, 'Japan', 'Norway')

In [87]:
plot_match(3893805, 'Japan', 'Costa Rica')

In [88]:
plot_match(3893793, 'Japan', 'Zambia')

In [19]:
Japan_matches_2022 = []
for item in json_data_2022:
    if item['home_team']['home_team_id'] == 778:
        Japan_matches_2022.append(item['match_id'])
    if item['away_team']['away_team_id'] == 778:
        Japan_matches_2022.append(item['match_id'])

In [20]:
for match_id in Japan_matches_2022:
    print(match_id)
    home = [match['home_team']['home_team_name'] for match in json_data_2022 if match['match_id'] == match_id][0]
    away = [match['away_team']['away_team_name'] for match in json_data_2022 if match['match_id'] == match_id][0]
    print(home, away)

3869219
Japan Croatia
3857295
Japan Costa Rica
3857284
Germany Japan
3857255
Japan Spain


In [21]:
url = f'https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/3857255.json'
match_events = load_json(url)

In [22]:
team = 'Japan'

In [23]:
starting_XI = [event for event in match_events if event['team']['name'] == team and event['type']['id'] == 35]
position_ids = [player['position']['id'] for player in starting_XI[0]['tactics']['lineup']]

In [24]:
tactic_XI = [event for event in match_events if event['team']['name'] == team and event['type']['id'] == 36]

In [25]:
team_events = [event for event in match_events if event['team']['name'] == team]

In [26]:
goal_events = [event for event in match_events if event['type']['id'] == 16 and 
        event['team']['name'] == team and event['shot']['outcome']['name'] == 'Goal']
no_goal_events = [event for event in match_events if event['type']['id'] == 16 and 
        event['team']['name'] == team and event['shot']['outcome']['name'] != 'Goal']

In [27]:
goal_events

[{'id': '7a8d09b4-e1da-4112-a8fc-6784cf2029a4',
  'index': 2476,
  'period': 2,
  'timestamp': '00:02:48.337',
  'minute': 47,
  'second': 48,
  'type': {'id': 16, 'name': 'Shot'},
  'possession': 68,
  'possession_team': {'id': 778, 'name': 'Japan'},
  'play_pattern': {'id': 1, 'name': 'Regular Play'},
  'team': {'id': 778, 'name': 'Japan'},
  'player': {'id': 8361, 'name': 'Ritsu Doan'},
  'position': {'id': 17, 'name': 'Right Wing'},
  'location': [100.5, 55.8],
  'duration': 0.80211,
  'related_events': ['31357970-dd9e-4523-a4d1-d3d354959e88'],
  'shot': {'statsbomb_xg': 0.027777622,
   'end_location': [120.0, 42.2, 1.9],
   'key_pass_id': 'f72a9178-aca5-41f9-8dc9-1a518332e5ec',
   'body_part': {'id': 38, 'name': 'Left Foot'},
   'type': {'id': 87, 'name': 'Open Play'},
   'outcome': {'id': 97, 'name': 'Goal'},
   'technique': {'id': 93, 'name': 'Normal'},
   'freeze_frame': [{'location': [107.5, 54.3],
     'player': {'id': 30628, 'name': 'Daizen Maeda'},
     'position': {'id': 2

In [33]:
def plot_match_m(match_id, team, opponent):
    url = f'https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/{match_id}.json'
    match_events = load_json(url)
    starting_XI = [event for event in match_events if event['team']['name'] == team and event['type']['id'] == 35]
    position_ids = [player['position']['id'] for player in starting_XI[0]['tactics']['lineup']]

    tactic_XI = [event for event in match_events if event['team']['name'] == team and event['type']['id'] == 36]

    team_events = [event for event in match_events if event['team']['name'] == team]

    goal_events = [event for event in match_events if event['type']['id'] == 16 and 
            event['team']['name'] == team and event['shot']['outcome']['name'] == 'Goal']
    no_goal_events = [event for event in match_events if event['type']['id'] == 16 and 
            event['team']['name'] == team and event['shot']['outcome']['name'] != 'Goal']

    goal_seq = {}
    for event in goal_events:
        before_goal_events = team_events[team_events.index(event) - 4:team_events.index(event)+1]
        before_goal_events= [e for e in before_goal_events if 'location' in e]
        goal_seq[event['index']] = before_goal_events

    no_goal_seq = {}
    for event in no_goal_events:
        before_no_goal_events = team_events[team_events.index(event) - 4:team_events.index(event)+1]
        before_no_goal_events= [e for e in before_no_goal_events if 'location' in e]
        no_goal_seq[event['index']] = before_no_goal_events

    dispo_events = [event for event in match_events if event['team']['name'] == opponent and
               event['type']['id'] == 3]

    field_layout = soccerfield.get_layout()
    field_layout.update(legend=dict(x=0.85, y=1))

    fig = go.Figure(layout=field_layout)
    fig.update_layout(title = dict(text = f"{team} Men's vs {opponent} Men's", x =0.5, y =0.9), 
                      title_font=dict(size=20, family='Arial', color='black'))

    # Starting lineups
    fig.add_trace(go.Scatter(
        x = [position_dict[i][0] for i in position_ids],
        y = [position_dict[i][1] for i in position_ids],
        mode='markers',
        name = 'starting XI',
        marker=dict(size=8, symbol = 'circle', color='lightblue',
            line = dict(width = 1, color = 'darkblue')),
    ))

    # Tactical shifts
    if len(tactic_XI) != 0:
        for tac in tactic_XI:
            time = str(tac['minute']) + ':' + str(tac['second'])
            tac_ids = [player['position']['id'] for player in tac['tactics']['lineup']]
            if tac_ids != position_ids:
                fig.add_trace(go.Scatter(
                    x = [position_dict[i][0] for i in tac_ids],
                    y = [position_dict[i][1] for i in tac_ids],
                    mode='markers',
                    name = f'tactical shit - {time}',
                    marker=dict(size=8, symbol = 'circle', color='lightblue', opacity=0.5,
                        line = dict(width = 1, color = 'steelblue')),
                    showlegend = True, visible='legendonly'
                ))

    # Shot with goals
    fig.add_trace(go.Scatter(
        x = [event['location'][0] for event in goal_events],
        y = [event['location'][1] for event in goal_events],
        legendgroup = 'goal shots',
        name = 'goal',
        mode='markers',
        marker=dict(size=6, symbol = 'circle', color='purple')
    ))

    # Shot with no goal
    fig.add_trace(go.Scatter(
        x = [event['location'][0] for event in no_goal_events],
        y = [event['location'][1] for event in no_goal_events],
        legendgroup = 'no goal shots',
        name = 'no goal',
        mode='markers',
        marker=dict(size=5, symbol = 'circle', color='darkgoldenrod')
    ))

    # Moves before shot with goals
    for key, seq in goal_seq.items():
        fig.add_trace(go.Scatter(
            x = [event['location'][0] for event in seq[:-1]],
            y = [event['location'][1] for event in seq[:-1]],
            legendgroup = 'goal shots',
            showlegend = False,
            mode='markers',
            marker=dict( size=6, symbol = 'circle', color='purple', opacity=0.3)
        ))
    for key, seq in goal_seq.items():
        fig.add_trace(go.Scatter(
            x=[event['location'][0] for event in seq],  
            y=[event['location'][1] for event in seq], 
            legendgroup = 'goal shots',
            showlegend = False,
            mode='lines',
            line=dict(color='purple', width = 0.25)
        ))

    # Moves before shot with no goal
    for key, seq in no_goal_seq.items():
        fig.add_trace(go.Scatter(
            x = [event['location'][0] for event in seq[:-1]],
            y = [event['location'][1] for event in seq[:-1]],
            legendgroup = 'no goal shots',
            showlegend = False,
            mode='markers',
            marker=dict( size=5, symbol = 'circle', color='tan', opacity=0.3)
        ))
    for key, seq in no_goal_seq.items():
        fig.add_trace(go.Scatter(
            x=[event['location'][0] for event in seq],  
            y=[event['location'][1] for event in seq], 
            legendgroup = 'no goal shots',
            showlegend = False,
            mode='lines',
            line=dict(color='tan', width = 0.18)
        ))    

    # Opponent disposession
    fig.add_trace(go.Scatter(
        x = [120-event['location'][0] for event in dispo_events],
        y = [event['location'][1] for event in dispo_events],
        name = 'opponent disposession',
        mode='markers',
        marker=dict(size=6, symbol = 'x', color='darkseagreen', opacity=0.6)
    ))

    fig.show()    

In [34]:
plot_match_m(3857255, 'Japan', 'Spain')

In [35]:
plot_match_m(3857255, 'Spain', 'Japan')

In [37]:
plot_match_m(3869219, 'Japan', 'Croatia')

In [38]:
plot_match_m(3857284, 'Japan', 'Germany')

In [39]:
plot_match_m(3857295, 'Japan', 'Costa Rica')